In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8')


In [ ]:
confirmed = pd.read_csv("covid19_Confirmed_dataset.csv")
deaths = pd.read_csv("covid19_deaths_dataset.csv")
happiness = pd.read_csv("worldwide_happiness_report.csv")
print("Confirmed Cases:")
display(confirmed.head())
print("\nDeaths:")
display(deaths.head())
print("\nHappiness Report:")
display(happiness.head())

In [ ]:
confirmed = confirmed.drop(columns=['Province/State', 'Lat', 'Long'], errors='ignore')
deaths = deaths.drop(columns=['Province/State', 'Lat', 'Long'], errors='ignore')
confirmed = confirmed.groupby('Country/Region').sum().reset_index()
deaths = deaths.groupby('Country/Region').sum().reset_index()
confirmed.rename(columns={'Country/Region': 'Country'}, inplace=True)
deaths.rename(columns={'Country/Region': 'Country'}, inplace=True)
latest_confirmed = confirmed[['Country', confirmed.columns[-1]]]
latest_deaths = deaths[['Country', deaths.columns[-1]]]
latest_confirmed.columns = ['Country', 'TotalConfirmed']
latest_deaths.columns = ['Country', 'TotalDeaths']
covid_df = pd.merge(latest_confirmed, latest_deaths, on='Country')
covid_df.head()


In [ ]:
print("COVID Summary:")
display(covid_df.describe())
top_confirmed = covid_df.sort_values(by='TotalConfirmed', ascending=False).head(10)
plt.figure(figsize=(10,5))
sns.barplot(x='TotalConfirmed', y='Country', data=top_confirmed, palette='Reds_r')
plt.title("Top 10 Countries by COVID-19 Confirmed Cases")
plt.xlabel("Total Confirmed Cases")
plt.ylabel("Country")
plt.show()

In [ ]:
happiness.columns = [col.strip().replace(' ', '_') for col in happiness.columns]
merged_df = pd.merge(happiness, covid_df, on='Country', how='inner')
merged_df.head()


In [ ]:
corr = merged_df[['Score', 'GDP_per_capita', 'Social_support', 'Healthy_life_expectancy',
                  'Freedom_to_make_life_choices', 'Generosity', 'Perceptions_of_corruption',
                  'TotalConfirmed', 'TotalDeaths']].corr()

plt.figure(figsize=(10,8))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Correlation Between Happiness Factors and COVID-19 Impact")
plt.show()


In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
sns.scatterplot(x='GDP_per_capita', y='TotalDeaths', data=merged_df)
plt.title("GDP vs COVID-19 Deaths")
plt.subplot(1,2,2)
sns.scatterplot(x='Score', y='TotalDeaths', data=merged_df)
plt.title("Happiness Score vs COVID-19 Deaths")
plt.tight_layout()
plt.show()

In [ ]:
insight = merged_df.sort_values(by=['Score', 'TotalDeaths'], ascending=[False, True]).head(10)
print("Countries with High Happiness & Low Deaths:")
display(insight[['Country', 'Score', 'TotalDeaths', 'GDP_per_capita']])
